<a href="https://colab.research.google.com/github/radhakrishnan-omotec/wildlife-repo/blob/main/2_Arduino_Wildlife_Detection_through_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id='step1'></a>
## Step 1: Import Dataset and Libraries

Our fist step is to import all the libraries used in this project.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from glob import glob
import pandas as pd
import os
import re
import random
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
from skimage import io
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
from torch.utils.data import (Dataset, DataLoader)  # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torchvision.models as models

In [6]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print ("cuda availability : ",use_cuda)

cuda availability :  True


In [7]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 224 x 224 x 3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 122 x 122 x 16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 56 x 56 x 32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # convolutional layer (sees 28 x 28 x 64 tensor)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        # convolutional layer (sees 14 x 14 x 128 tensor)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)

        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

        self.conv_bn1 = nn.BatchNorm2d(224,3)
        self.conv_bn2 = nn.BatchNorm2d(16)
        self.conv_bn3 = nn.BatchNorm2d(32)
        self.conv_bn4 = nn.BatchNorm2d(64)
        self.conv_bn5 = nn.BatchNorm2d(128)
        self.conv_bn6 = nn.BatchNorm2d(256)

        # linear layer (64 * 4 * 4 -> 133)
        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        # linear layer (133 -> 133)
        self.fc2 = nn.Linear(512, 20)


    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.conv_bn2(self.pool(F.relu(self.conv1(x))))
        x = self.conv_bn3(self.pool(F.relu(self.conv2(x))))
        x = self.conv_bn4(self.pool(F.relu(self.conv3(x))))
        x = self.conv_bn5(self.pool(F.relu(self.conv4(x))))
        x = self.conv_bn6(self.pool(F.relu(self.conv5(x))))
        # flatten image input
        x = x.view(-1, 256 * 7 * 7)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()
print(model_scratch)

# move tensors to GPU if CUDA is available
if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (conv_bn1): BatchNorm2d(224, eps=3, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_bn6

### LOAD MODEL WITHOUT TRAINING

In [8]:

# load the model that got the best validation accuracy
model_scratch.load_state_dict(torch.load('/content/gdrive/MyDrive/WILDLIFE-PROJECT/model_scratch_wildlife.pt'))

<All keys matched successfully>

---

<a id='step-LAST'></a>
## Complete Code to Capture input image from Webcam and then Predict the category

---

<a id='step-START'></a>
## Start of Testing using Webcam

In [9]:
import cv2
from google.colab.output import eval_js
from IPython.display import display, Javascript, Image
import numpy as np
import shutil
import base64
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision import transforms
from torch.autograd import Variable
import random
import re
from PIL import Image

def find_closest_object(contours, min_area=100):
    """
    Finds the contour with the largest area among contours with area greater than min_area.

    Args:
        contours: List of contours.
        min_area: Minimum area threshold.

    Returns:
        The contour with the largest area among contours with area greater than min_area.
    """
    max_area = min_area
    closest_contour = None

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area and area > max_area:
            max_area = area
            closest_contour = contour

    return closest_contour

def draw_contour_box(image, contour):
    """
    Draws a bounding box around the contour on the image.

    Args:
        image: Input image.
        contour: Contour to draw the bounding box around.

    Returns:
        The image with the bounding box drawn.
    """
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return image

def take_photo_and_predict(filename='photo.jpg', quality=0.8):
    # Take photo with box drawn
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))
    # Convert JS image data to OpenCV format
    img = cv2.imdecode(np.frombuffer(base64.b64decode(data.split(',')[1]), np.uint8), cv2.IMREAD_COLOR)

    # Convert the image to grayscale for contour detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to remove noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # Threshold the image
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Find the contour with the largest area
    closest_contour = find_closest_object(contours)

    # If a contour is found, draw a bounding box around it and predict breed
    if closest_contour is not None:
        img_with_box = draw_contour_box(img.copy(), closest_contour)

        # Save image with box drawn
        cv2.imwrite(filename, img_with_box)
        print('Saved to {}'.format(filename))

        # Predict breed
        img = Image.open(filename)
        predicted_breed, true_breed = predict_breed(img)
        print("Predicted Animal:", predicted_breed, "\nTrue Animal:", true_breed)

        # Display the image
        imgplot = plt.imshow(img)
        plt.show()
    else:
        print('No objects detected.')
        return None

    return filename

def predict_breed(img):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )

    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize]
    )

    img_tensor = preprocess(img).float()
    img_tensor.unsqueeze_(0)  # Insert the new axis at index 0 i.e. in front of the other axes/dims.

    if use_cuda:
        img_tensor = img_tensor.cuda()

    model_scratch.eval()
    output = model_scratch(img_tensor)  # Returns a Tensor of shape (batch, num class labels)

    # Our prediction will be the index of the class label with the largest value.
    predict_index = output.data.cpu().numpy().argmax()

    predicted_breed = class_names[predict_index]
    true_breed = image_datasets['train'].classes[predict_index]

    return (predicted_breed, true_breed)

try:
    filename = take_photo_and_predict('/content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')  # Saving to Google Drive
    if filename:
        print('Moved to /content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')
        # Move the file to the desired location
        shutil.move(filename, '/content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))

<IPython.core.display.Javascript object>

Saved to /content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg
name 'class_names' is not defined


<a id='step-END'></a>
## End of Testing

In [ ]:
import cv2
from google.colab.output import eval_js
from IPython.display import display, Javascript, Image
import numpy as np
import shutil
import base64
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from torchvision import transforms
from torch.autograd import Variable
import random
import re
from PIL import Image

# Load the XYZ.h5 model
import h5py
import torch
import torch.nn as nn

# Define the CNN architecture class
class XYZModel(nn.Module):
    def __init__(self):
        super(XYZModel, self).__init__()
        # Define the architecture of the XYZ model
        # Example:
        # self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # self.pool = nn.MaxPool2d(2, 2)
        # ...

    def forward(self, x):
        # Define the forward pass of the XYZ model
        pass

# Load the XYZ model
xyz_model = XYZModel()
xyz_model.load_state_dict(torch.load('/path/to/XYZ.h5'))
if torch.cuda.is_available():
    xyz_model.cuda()

# Define other functions and variables used in the code

def find_closest_object(contours, min_area=100):
    # Function definition remains unchanged

def draw_contour_box(image, contour):
    # Function definition remains unchanged

def take_photo_and_predict(filename='photo.jpg', quality=0.8):
    # Function definition remains unchanged

def predict_breed(img):
    # Function definition remains unchanged

try:
    filename = take_photo_and_predict('/content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')
    if filename:
        print('Moved to /content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')
        shutil.move(filename, '/content/gdrive/MyDrive/WILDLIFE-PROJECT/input_photo1.jpg')
except Exception as err:
    print(str(err))